In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# Import data from google drive
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
file_list = drive.ListFile({'q': "'1RxNKaI-lPF6L0jIx4hDimtsh1jNN3a3u' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: notMNIST.pickle, id: 1rymPKySp1y2D8QS-7-8WqG57k98wk0_Q


In [0]:
# Download req. pickle file
downloaded = drive.CreateFile({'id': '1rymPKySp1y2D8QS-7-8WqG57k98wk0_Q'})
downloaded.GetContentFile('notMNIST.pickle')

In [0]:
import pickle
import numpy as np

In [6]:
with open('notMNIST.pickle', 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [10]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD

In [15]:
# Set up the model
model = Sequential()

# Add the first layer
model.add(Dense(1024, activation = 'relu', input_shape = (784, )))

# Add the output layer
model.add(Dense(10, activation='softmax'))

# Create SGD optimizer with specified learning rate: my_optimizer
my_optimizer = SGD()
  

# Compile the model
model.compile(optimizer = my_optimizer, metrics = ['accuracy'], loss='categorical_crossentropy')

# Fit the model
with tf.device('/gpu:0'):
  model.fit(train_dataset, train_labels, epochs=10, batch_size=64, validation_data=(valid_dataset, valid_labels))

Train on 200000 samples, validate on 10000 samples
Epoch 1/10
200000/200000 [==============================] - 17s 84us/step - loss: 0.6616 - acc: 0.8189 - val_loss: 0.5415 - val_acc: 0.8454
Epoch 2/10
118528/200000 [================>.............] - ETA: 6s - loss: 0.5397 - acc: 0.8472

200000/200000 [==============================] - 17s 84us/step - loss: 0.5310 - acc: 0.8492 - val_loss: 0.5004 - val_acc: 0.8557
Epoch 3/10
200000/200000 [==============================] - 17s 85us/step - loss: 0.4949 - acc: 0.8582 - val_loss: 0.4751 - val_acc: 0.8621
Epoch 4/10
 28672/200000 [===>..........................] - ETA: 14s - loss: 0.4762 - acc: 0.8646

200000/200000 [==============================] - 17s 84us/step - loss: 0.4686 - acc: 0.8652 - val_loss: 0.4552 - val_acc: 0.8660
Epoch 5/10
200000/200000 [==============================] - 17s 84us/step - loss: 0.4469 - acc: 0.8718 - val_loss: 0.4381 - val_acc: 0.8716
Epoch 6/10
  7488/200000 [>.............................] - ETA: 16s - loss: 0.4456 - acc: 0.8711

200000/200000 [==============================] - 17s 84us/step - loss: 0.4285 - acc: 0.8770 - val_loss: 0.4251 - val_acc: 0.8769
Epoch 7/10
200000/200000 [==============================] - 17s 84us/step - loss: 0.4125 - acc: 0.8812 - val_loss: 0.4146 - val_acc: 0.8798
Epoch 8/10
  3840/200000 [..............................] - ETA: 16s - loss: 0.4203 - acc: 0.8742

200000/200000 [==============================] - 17s 84us/step - loss: 0.3986 - acc: 0.8854 - val_loss: 0.4042 - val_acc: 0.8852
Epoch 9/10
200000/200000 [==============================] - 17s 84us/step - loss: 0.3861 - acc: 0.8894 - val_loss: 0.3973 - val_acc: 0.8842
Epoch 10/10
  3328/200000 [..............................] - ETA: 16s - loss: 0.3854 - acc: 0.8837

200000/200000 [==============================] - 17s 84us/step - loss: 0.3748 - acc: 0.8925 - val_loss: 0.3894 - val_acc: 0.8867


In [16]:
# Score trained model.
scores = model.evaluate(test_dataset, test_labels, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 1s 62us/step
Test loss: 0.21248694378733635
Test accuracy: 0.9399
